In [ ]:
!pip install wbgapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import wbgapi as wb

In [ ]:
class Indicators:
  def __init__(self):
    self.all_indicators = ['NY.GDP.PCAP.PP.CD', 'FP.CPI.TOTL.ZG', 'SL.UEM.TOTL.ZS', 'BX.KLT.DINV.CD.WD',    'NV.AGR.TOTL.ZS', 'NE.IMP.GNFS.ZS', 'NE.EXP.GNFS.ZS', 'SP.DYN.LE00.IN', 'SH.IMM.IDPT',    'SH.DYN.NMRT', 'SH.HIV.1524.MA.ZS', 'SP.DYN.CDRT.IN', 'SP.DYN.TFRT.IN', 'SM.POP.NETM',    'SP.POP.0014.TO.ZS', 'SP.POP.65UP.TO.ZS', 'SP.POP.TOTL.FE.ZS', 'SM.POP.REFG.OR',    'SP.POP.GROW', 'SP.POP.TOTL', 'SP.URB.TOTL.IN.ZS']
    self.economic_indicators = ['NY.GDP.PCAP.PP.CD', 'FP.CPI.TOTL.ZG', 'SL.UEM.TOTL.ZS', 'BX.KLT.DINV.CD.WD',    'NV.AGR.TOTL.ZS', 'NE.IMP.GNFS.ZS', 'NE.EXP.GNFS.ZS']
    self.health_indicators = ['SP.DYN.LE00.IN', 'SH.IMM.IDPT',    'SH.DYN.NMRT', 'SH.HIV.1524.MA.ZS']
    self.sociodemographic_indicators = ['SP.DYN.CDRT.IN', 'SP.DYN.TFRT.IN', 'SM.POP.NETM',    'SP.POP.0014.TO.ZS', 'SP.POP.65UP.TO.ZS', 'SP.POP.TOTL.FE.ZS', 'SM.POP.REFG.OR',    'SP.POP.GROW', 'SP.POP.TOTL', 'SP.URB.TOTL.IN.ZS']
    self.__sociodemographic_indicators_full_name = ['Death rate, crude (per 1,000 people)', '	Fertility rate, total (births per woman)', 'Net migration', 'Population ages 0-14 (% of total population)', 'Population ages 65 and above (% of total population)', 'Population growth (annual %)', 'Population, female (% of total population)', 'Population, total', 'Refugee population by country or territory of origin', 'Urban population (% of total population)']
    self.__health_indicators_full_name = ['Immunization, DPT (% of children ages 12-23 months)', 'Life expectancy at birth, total (years)', '	Mortality rate, neonatal (per 1,000 live births)', 'Prevalence of HIV, male (% ages 15-24)']
    self.__economic_indicators_full_name = ['Agriculture, forestry, and fishing, value added (% of GDP)', 'Exports of goods and services (% of GDP)', 'Foreign direct investment, net inflows (BoP, current US$)', 'GDP per capita, PPP (current international $)', 'Imports of goods and services (% of GDP)', '	Inflation, consumer prices (annual %)', 'Unemployment, total (% of total labor force) (modeled ILO estimate)']
    

  def get_sociodemographic_indicators_full_name(self):
        return self.__sociodemographic_indicators_full_name
  
  def get_health_indicators_full_name(self):
        return self.__health_indicators_full_name
  
  def get_economic_indicators_full_name(self):
        return self.__economic_indicators_full_name
    
   

In [ ]:
class Countries:
  def __init__(self):
    self.all_countries = ['AFE', 'AFW', 'ALB', 'AZE', 'BDI', 'BEN', 'BFA', 'BGD', 'BGR', 'BLR', 'BOL', 'BTN', 'BWA', 'CHL', 'CIV', 'CMR', 'COD', 'COG', 'COL', 'CRI', 'DOM', 'ECU', 'EGY', 'ESP', 'FCS', 'FJI', 'FRA', 'GAB', 'GEO', 'GHA', 'GMB', 'GNB', 'GRC', 'GTM', 'HND', 'HPC', 'HRV', 'HTI', 'IDA', 'IDB', 'IDN', 'IDX', 'IRN', 'ITA', 'JAM', 'KAZ', 'KEN', 'KGZ', 'KHM', 'LAO', 'LIC', 'LKA', 'LTU', 'LVA', 'MAR', 'MDA', 'MDG', 'MEA', 'MEX', 'MLI', 'MNA', 'MNG', 'MRT', 'MUS', 'MYS', 'NER', 'NGA', 'NLD', 'NPL', 'PAK', 'PER', 'PHL', 'PRE', 'PRT', 'PRY', 'ROU', 'RWA', 'SDN', 'SEN', 'SGP', 'SLV', 'SSA', 'SSF', 'SVK', 'SVN', 'SWZ', 'TCD', 'TGO', 'THA', 'TMN', 'TSS', 'TUN', 'TZA', 'UGA', 'UKR', 'URY', 'VNM', 'WLD', 'ZAF', 'ZMB']
    self.africa_countries = ['AFE', 'AFW', 'BDI', 'BEN', 'BFA', 'BWA', 'CIV', 'CMR', 'COD', 'COG', 'DJI', 'DZA', 'EGY', 'ERI', 'ESH', 'ETH', 'GAB', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'KEN', 'LBR', 'LBY', 'LSO', 'MAR', 'MDG', 'MLI', 'MOZ', 'MRT', 'MUS', 'MWI', 'NAM', 'NER', 'NGA', 'RWA', 'SDN', 'SEN', 'SLE', 'SOM', 'SSD', 'STP', 'SWZ', 'SYC', 'TCD', 'TGO', 'TUN', 'TZA', 'UGA', 'ZAF', 'ZMB', 'ZWE']
    self.asia_countries = ['AZE', 'BGD', 'BTN', 'CHN', 'IDN', 'IND', 'IRN', 'IRQ', 'ISR', 'JOR', 'KAZ', 'KGZ', 'KHM', 'KOR', 'KWT', 'LAO', 'LBN', 'LKA', 'MDA', 'MYS', 'NPL', 'OMN', 'PAK', 'PHL', 'PRK', 'PSE', 'QAT', 'SAU', 'SGP', 'SYR', 'THA', 'TJK', 'TKM', 'TLS', 'TUR', 'UZB', 'VNM', 'YEM']
    self.europe_countries = ['ALB', 'AND', 'AUT', 'BEL', 'BGR', 'BIH', 'BLR', 'CHE', 'CYP', 'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GEO', 'GRC', 'HRV', 'HUN', 'IRL', 'ISL', 'ITA', 'KOS', 'LIE', 'LTU', 'LUX', 'LVA', 'MDG', 'MKD', 'MLT', 'MNE', 'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SMR', 'SRB', 'SVK', 'SVN', 'SWE', 'UKR', 'VAT']
    self.north_america_countries = ['ATG', 'BHS', 'BLZ', 'BRB', 'CAN', 'CRI', 'CUB', 'DOM', 'GRD', 'GTM', 'HND', 'HTI', 'JAM', 'MEX', 'NIC', 'PAN', 'SLV', 'TTO', 'USA']
    self.south_america_countries = ['ARG', 'BOL', 'BRA', 'CHL', 'COL', 'ECU', 'GUY', 'PER', 'PRY', 'SUR', 'URY', 'VEN']
    self.oceania_countries = ['ASM', 'AUS', 'COK', 'FJI', 'KIR', 'MHL', 'FSM']

In [ ]:
x = Indicators()
wb.series.info(x.economic_indicators)

id,value
NV.AGR.TOTL.ZS,"Agriculture, forestry, and fishing, value added (% of GDP)"
NE.EXP.GNFS.ZS,Exports of goods and services (% of GDP)
BX.KLT.DINV.CD.WD,"Foreign direct investment, net inflows (BoP, current US$)"
NY.GDP.PCAP.PP.CD,"GDP per capita, PPP (current international $)"
NE.IMP.GNFS.ZS,Imports of goods and services (% of GDP)
FP.CPI.TOTL.ZG,"Inflation, consumer prices (annual %)"
SL.UEM.TOTL.ZS,"Unemployment, total (% of total labor force) (modeled ILO estimate)"
,7 elements


In [ ]:
class Helper_functions(Indicators, Countries):
  
  def __init__(self):
    super().__init__()

  def data_frame(self, indicators:Indicators, countries:Countries):
    socio_dict = dict(zip(self.sociodemographic_indicators ,self.get_sociodemographic_indicators_full_name))
    health_dict = dict(zip(self.health_indicators,self.get_health_indicators_full_name))
    economic_dict = dict(zip(self.economic_indicators,self.get_economic_indicators_full_name))
    mappings = socio_dict | health_dict | economic_dict
    df = wb.data.DataFrame(indicators, economy=countries, time=range(1990,2021), labels=True)
    df = df.rename(columns=lambda x: x.replace('YR', ''))
    id_vars=[f'{x}' for x in range(1990,2021)]

    df_melted = df.reset_index(level=['economy','series'])
    df_melted['series_name'] = df_melted['series'].replace(mappings)
    return df_melted

In [ ]:
help = Helper_functions()
countries = Countries()
indicators = Indicators()
#df = help.data_frame(indicators.economic_indicators, countries.all_countries)
df = wb.data.DataFrame(indicators.all_indicators, countries.all_countries, time=range(1990,2021))
df

YR1990        YR1991        YR1992  \
economy series                                                        
AFE     BX.KLT.DINV.CD.WD  8.867306e+07  1.241969e+09  6.523250e+08   
        FP.CPI.TOTL.ZG     1.245791e+01  1.767810e+01  1.616761e+01   
        NE.EXP.GNFS.ZS     2.061089e+01  1.875437e+01  2.183226e+01   
        NE.IMP.GNFS.ZS     1.966854e+01  1.948130e+01  2.119271e+01   
        NV.AGR.TOTL.ZS     1.751717e+01  1.898274e+01  1.248433e+01   
...                                 ...           ...           ...   
ZMB     SP.POP.65UP.TO.ZS  2.692339e+00  2.661017e+00  2.626364e+00   
        SP.POP.GROW        2.571365e+00  2.493438e+00  2.430372e+00   
        SP.POP.TOTL        7.686401e+06  7.880466e+06  8.074337e+06   
        SP.POP.TOTL.FE.ZS  5.168522e+01  5.173123e+01  5.177919e+01   
        SP.URB.TOTL.IN.ZS  3.940700e+01  3.898900e+01  3.851400e+01   

                                 YR1993        YR1994        YR1995  \
economy series                                                        
AFE     BX.KLT.DINV.CD.WD  8.395097e+08  1.187865e+09  2.662959e+09   
        FP.CPI.TOTL.ZG     1.313566e+01  1.485281e+01  1.228859e+01   
        NE.EXP.GNFS.ZS     2.308516e+01  2.412068e+01  2.479106e+01   
        NE.IMP.GNFS.ZS     2.138473e+01  2.413653e+01  2.599087e+01   
        NV.AGR.TOTL.ZS     1.279103e+01  1.232410e+01  1.200164e+01   
...                                 ...           ...           ...   
ZMB     SP.POP.65UP.TO.ZS  2.588322e+00  2.546236e+00  2.500577e+00   
        SP.POP.GROW        2.405463e+00  2.428276e+00  2.446966e+00   
        SP.POP.TOTL        8.270917e+06  8.474216e+06  8.684135e+06   
        SP.POP.TOTL.FE.ZS  5.182728e+01  5.186751e+01  5.189972e+01   
        SP.URB.TOTL.IN.ZS  3.804200e+01  3.757200e+01  3.710400e+01   

                                 YR1996        YR1997        YR1998  \
economy series                                                        
AFE     BX.KLT.DINV.CD.WD  2.087427e+09  5.757028e+09  4.121574e+09   
        FP.CPI.TOTL.ZG     9.706586e+00  1.024960e+01  7.495256e+00   
        NE.EXP.GNFS.ZS     2.594899e+01  2.513802e+01  2.522770e+01   
        NE.IMP.GNFS.ZS     2.660097e+01  2.629356e+01  2.720031e+01   
        NV.AGR.TOTL.ZS     1.251463e+01  1.314760e+01  1.326676e+01   
...                                 ...           ...           ...   
ZMB     SP.POP.65UP.TO.ZS  2.451815e+00  2.399242e+00  2.344840e+00   
        SP.POP.GROW        2.478031e+00  2.563320e+00  2.586109e+00   
        SP.POP.TOTL        8.902019e+06  9.133156e+06  9.372430e+06   
        SP.POP.TOTL.FE.ZS  5.192238e+01  5.192309e+01  5.191314e+01   
        SP.URB.TOTL.IN.ZS  3.663800e+01  3.617600e+01  3.571600e+01   

                                 YR1999  ...        YR2011        YR2012  \
economy series                           ...                               
AFE     BX.KLT.DINV.CD.WD  6.254804e+09  ...  1.753871e+10  2.463634e+10   
        FP.CPI.TOTL.ZG     7.819865e+00  ...  8.971206e+00  9.158707e+00   
        NE.EXP.GNFS.ZS     2.515532e+01  ...  3.119676e+01  2.967840e+01   
        NE.IMP.GNFS.ZS     2.566947e+01  ...  3.179584e+01  3.296149e+01   
        NV.AGR.TOTL.ZS     1.231437e+01  ...  1.050683e+01  1.144093e+01   
...                                 ...  ...           ...           ...   
ZMB     SP.POP.65UP.TO.ZS  2.288775e+00  ...  1.726035e+00  1.708656e+00   
        SP.POP.GROW        2.620055e+00  ...  3.377110e+00  3.301480e+00   
        SP.POP.TOTL        9.621238e+06  ...  1.426581e+07  1.474466e+07   
        SP.POP.TOTL.FE.ZS  5.189251e+01  ...  5.089585e+01  5.086026e+01   
        SP.URB.TOTL.IN.ZS  3.525800e+01  ...  3.985100e+01  4.035400e+01   

                                 YR2013        YR2014        YR2015  \
economy series                                                        
AFE     BX.KLT.DINV.CD.WD  2.153425e+10  2.768142e+10  2.877423e+10   
        FP.CPI.TOTL.ZG     5.750981e+00  5.370290e+00  5.25

Oto objaśnienia wybranych wskaźników: <br>
**Wskaźniki ekonomiczne** <br>
1. 'NY.GDP.PCAP.PP.CD': Wskaźnik PKB na mieszkańca w parze siły nabywczej - mierzy wartość produkcji towarów i usług przypadającą na jednego mieszkańca, uwzględniając różnice w siłach nabywczych walut między krajami.
2. 'FP.CPI.TOTL.ZG': Wskaźnik inflacji - mierzy wzrost średniego poziomu cen w gospodarce i jest używany do monitorowania zmiany siły nabywczej waluty.
3. 'SL.UEM.TOTL.ZS': Wskaźnik bezrobocia - mierzy odsetek osób bezrobotnych w wieku 15 lat i starszych w stosunku do siły roboczej.
4. 'BX.KLT.DINV.CD.WD': Wskaźnik bezpośrednich inwestycji zagranicznych - mierzy wartość bezpośrednich inwestycji dokonywanych przez inwestorów zagranicznych w kraju.
5. 'NV.AGR.TOTL.ZS': Udział sektora rolnego w PKB - mierzy wartość produkcji rolniczej w stosunku do wartości całkowitej produkcji w kraju.
6. 'NE.IMP.GNFS.ZS': Wskaźnik importu towarów i usług - mierzy wartość importu w stosunku do wartości całkowitej produkcji w kraju.
7. 'NE.EXP.GNFS.ZS': Wskaźnik eksportu towarów i usług - mierzy wartość eksportu w stosunku do wartości całkowitej produkcji w kraju.
**Wskaźniki zdrowotne** <br>
8. 'SH.DYN.NMRT': Wskaźnik śmiertelności niemowląt - liczba zgonów noworodków na 1000 urodzeń żywych. A
12. 'SH.IMM.IDPT': Udział dzieci w wieku 12-23 miesięcy, które otrzymały szczepionkę przeciwko błonicy, tężcowi i krztuścowi - wskaźnik ten mierzy odsetek dzieci w wieku 12-23 miesięcy, które otrzymały trzy dawki szczepionki przeciwko błonicy, tężcow. A
13. 'SP.DYN.LE00.IN': Oczekiwana długość życia (ang. Life expectancy at birth, total (years)) - jest to średnia liczba lat, jakie spodziewa się żyć osoba urodzona w danym kraju w danym roku. A
14. 'SH.HIV.1524.MA.ZS': Udział mężczyzn w wieku 15-24 lat zakażonych HIV (ang. HIV prevalence, male (% ages 15-24)) - jest to odsetek mężczyzn w wieku 15-24 lat, którzy są zakażeni wirusem HIV. A
**Wskaźniki socjodemograficzne** <br>
15. 'SP.DYN.CDRT.IN': Wskaźnik umieralności (ang. Mortality rate, crude (per 1,000 people)) - liczba zgonów na 1000 mieszkańców.
16. 'SP.DYN.TFRT.IN': Wskaźnik dzietności (ang. Fertility rate, total (births per woman)) - liczba dzieci, jakie przypada na kobietę w wieku rozrodczym.
17. 'SM.POP.NETM': Wskaźnik migracji netto (ang. Net migration) - to różnica między liczbą osób przybywających do danego kraju, a liczbą osób opuszczających ten kraj.
18. 'SP.POP.0014.TO.ZS': Udział populacji w wieku 0-14 lat (ang. Population, ages 0-14 (% of total population)) - to odsetek populacji w wieku od 0 do 14 lat w całkowitej populacji danego kraju.
19. 'SP.POP.65UP.TO.ZS': Udział populacji w wieku powyżej 65 lat (ang. Population ages 65 and above (% of total population)) - to odsetek populacji w wieku powyżej 65 lat w całkowitej populacji danego kraju.
20. 'SP.POP.TOTL.FE.ZS': Udział kobiet w populacji (ang. Population, female (% of total population)) - to odsetek kobiet w całkowitej populacji danego kraju.
21. 'SM.POP.REFG.OR': Liczba uchodźców (ang. Refugee
22. 'SP.POP.GROW': Tempo wzrostu populacji (ang. Population growth (annual %)) - to procentowa zmiana liczby ludności danego kraju w ciągu roku.
23. 'SP.POP.TOTL': Liczba ludności (ang. Population, total) - to liczba ludności w danym kraju.
24. 'SP.URB.TOTL.IN.ZS': Odsetek ludności mieszkającej w miastach (ang. Urban population (% of total population)) - to odsetek ludności mieszkającej w miastach w całkowitej populacji danego kraju.
25. 'SL.UEM.TOTL.ZS': Stopa bezrobocia (ang. Unemployment, total (% of total labor force)) - to odsetek osób bezrobotnych w całkowitej sile roboczej danego kraju.